In [61]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

## Code which scrapes the recently reviews (only 100 pages - 1,400 reviews)

In [62]:

def ratebeer_scraper(page_start=1, page_end=2, url_set=None):
    
    beer_name = []
    beer_rating = []
    brewery_name = []
    brewery_location = []
    review_content = []
    reviewer_name = []
 
    if url_set == None:
        print("Need url to run")
        return

    for x in tqdm(range(page_start, page_end + 1)):
        try:
            url_format = url_set.format(x)
            r = requests.get(url_format)
            soup = BeautifulSoup(r.text, 'html.parser')
            table = soup.find('table', attrs={'class': 'table'})
            table_rows = table.find_all('tr')

            for rows in table_rows:

            # Scrape Beer Name    
                try:
                    beer_name.append(rows.find('a', attrs={'style':'font-size:20px; font-weight:bold;'}).text)
                except:
                    beer_name.append(None)
            # Scrape Review Rating  
                try:
                    beer_rating.append(rows.find('span', attrs={'class':'uas'}).text)
                except:
                    beer_rating.append(None)
            # Scrape Brewery Name  
                try:
                    brewery_div = rows.find('div')
                    brewery_name.append(brewery_div.a.text)
                except:
                    brewery_name.append(None)
            # Scrape Brewery Location
                try:
                    brewery_location.append(rows.find('span', attrs={'class':'small'}).text.strip())
                except:
                    brewery_location.append(None)
            # Scrape Reviewer Name
                try:
                    reviewer_name.append(rows.find('span', attrs={'class':'pull-right'}).text.strip())
                except:
                    reviewer_name.append(None)
            # Scrape content
                try:
                    review_content.append(rows.find('div', attrs={'style':"color:#666;"}).text.strip())
                except:
                    review_content.append(None)
        except:
            pass

    beerz = pd.DataFrame({'beer_name': beer_name,
                         'beer_rating': beer_rating,
                        'brewery_name': brewery_name,
                         'brewery_location': brewery_location,
                        'review_content': review_content,
                        'reviewer_name': reviewer_name})
    return beerz



In [63]:
# Scraping most recent reviews
url = 'https://www.ratebeer.com/beer-ratings/0/{}/'
df_recent = ratebeer_scraper(0,100,url)

# # Scraping new beers
# url = 'https://www.ratebeer.com/beer-ratings/5/{}/'
# df_new = ratebeer_scraper(0,1,url)

# # Scraping fave beers
# url = 'https://www.ratebeer.com/beer-ratings/4/{}/'
# df_fave = ratebeer_scraper(0,1,url)

# # Scraping rave beers
# url = 'https://www.ratebeer.com/beer-ratings/1/{}/'
# df_rave = ratebeer_scraper(0,1,url)

# # Scraping rant beers
# url = 'https://www.ratebeer.com/beer-ratings/2/{}/'
# df_rant = ratebeer_scraper(0,1,url)

# # Scraping anomalous beers
# url = 'https://www.ratebeer.com/beer-ratings/3/{}/'
# df_anom = ratebeer_scraper(0,1,url)

In [64]:
# getting rid of non-ascii characters from the reviews as it breaks csv
try:
    df_rave.review_content = df_rave.review_content.apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
except:
    pass

In [65]:
# read current data from csv (there is a backup at all_reviews_backup.csv)
df = pd.read_csv('all_reviews.csv')

In [66]:
# concatenating all of the scrapes together with the old df
try:
    df = pd.concat([df, df_recent, df_new, df_fave, df_rave, df_rant, df_anom])
except:
    df = pd.concat([df, df_recent])
print(len(df))
df.drop_duplicates(inplace=True)
print(len(df))
df.beer_name = df.beer_name.apply(lambda x: x.strip())



15318
13846


In [67]:
df

,beer_name,beer_rating,brewery_location,brewery_name,review_content,reviewer_name
0,St. Peter's Ruby Red Ale,3.3,"Bungay, Suffolk",St. Peter's Brewery,"Ruby color, offwhite coam. Aroma and taste: fl...",alex_leit
1,Shepherd Neame Spitfire (Bottle),2.2,"Faversham, Kent",Shepherd Neame,Disappointing show from the lads at ‘Britain’s...,shivermetimber.gray
2,Arbor Rocketman,3.8,"Bristol, Gloucestershire",Arbor Ales,"Puszka. Mętne o żółtym kolorze. Piana biała,ob...",MarcinG
3,Shepherd Neame India Pale Ale (Bottle),3.4,"Faversham, Kent",Shepherd Neame,Bottle. Amber with bronze color and white foam...,alex_leit
4,Shepherd Neame Christmas Ale,3.3,"Faversham, Kent",Shepherd Neame,Bottle. Aroma: English hops and malts with tof...,alex_leit
...,...,...,...,...,...,...
1510,Funky Buddha Tequila Margarita Gose,3.7,"Oakland Park, Florida",Funky Buddha Brewery (Constellation Brands),"Bottle pour, cloudy amber. Aroma of banana, ch...",lavaaaron
1511,Odd Breed Wild Ales Extra Thai Rack,3.8,"Pompano Beach, Florida",Odd Breed Wild Ales,"750 ml bottle. Pours a mostly clear, straw gol...",juiceisloose
1512,Vibrant Forest / Emperor's Yub Nub,4.0,"Hardley, Hampshire",Vibrant Forest Brewery,330ml bottle from leftfieldbeer.co.uk. Very da...,coil
1513,Big Stump Llama Drama,3.8,"Sacramento, California",Big Stump Brewing Company,"Can from Corti Bros. Pours slightly dirty, ora...",bhensonb


In [68]:
df.to_csv('all_reviews.csv',index=False)
# df = pd.read_csv('all_reviews.csv')
not_found_df = pd.read_csv("not_found.csv")

## Get more information on all of the beers - Selenium

In [69]:
from selenium import webdriver
import time

In [70]:
# extract already scraped data from previous scrapes so don't rescrape
beer_df = pd.read_csv('beer_deets.csv')

In [71]:
# compare to the unique list to create a new to-be-scraped list
new_beer = df.beer_name[(~df.beer_name.isin(beer_df.name_found))&(~df.beer_name.isin(not_found_df.not_found))].unique()
print("There are", len(new_beer), "new beers to search for!")

There are 298 new beers to search for!


In [72]:
item_names = []
desc_string = []

driver = webdriver.Chrome(executable_path='/Users/lukebetham/Downloads/chromedriver')

for beer in tqdm(new_beer):
    driver.get("https://www.ratebeer.com/search?beername={}".format(beer))
    time.sleep(2)
    item_tags = driver.find_elements_by_class_name('fg-1')

    x=1
    beer_temp = []
    for item in item_tags:        
        if x <2:
            try:
                item_names.append(item.find_element_by_class_name('fd-r').text)
                x+=1
            except:
                pass
            try:
                beer_temp.append(item.text)
            except:
                pass
    try:
        desc_string.append(beer_temp[4])
    except:
        desc_string.append(None)

driver.quit()

In [73]:
beer_deets = pd.DataFrame(desc_string, columns = ['return_string'])
beer_deets

,return_string
0,Odyssey Kiss It\n🇬🇧 Pale Ale - American / APA ...
1,Black Sheep Venus & Mars\n🇬🇧 Pale Ale - Americ...
2,Crooked Stave L'Brett d'Or Grand Cru\n🇺🇸 Sour ...
3,Logo\nThe most comprehensive ratings and revie...
4,Brauhof Saar Hops Heaven One\n🇩🇪 IPA • 7.3%\n-...
...,...
293,Rorschach Malevolent Benevolence - Jelly Donut...
294,Rorschach Hedonism - Pineapple\n🇨🇦 Sour / Wild...
295,The Veil Hornswaggler - Baked Apple\n🇺🇸 Stout ...
296,Radical Road Salvation\n🇨🇦 Flavored - Fruit • ...


In [74]:
## cleaning the scraped data
beer_deets.dropna(inplace=True)
beer_deets = beer_deets[~beer_deets.return_string.str.startswith('Logo\n')].copy()
no_alcohol = beer_deets[~beer_deets.return_string.str.contains('%')].copy()
beer_deets = beer_deets[beer_deets.return_string.str.contains('%')].copy()


In [75]:
# sorting the scraped data into columns
beer_deets['name_found'] = beer_deets.return_string.apply(lambda x: x.split('\n')[0].strip())
beer_deets['flag'] = beer_deets.return_string.apply(lambda x: x.split('\n')[1].split('•')[0].split(" ")[0])
beer_deets['type'] = beer_deets.return_string.apply(lambda x: x.split('\n')[1].split('•')[0][2:])
beer_deets['abv'] = beer_deets.return_string.apply(lambda x: float(x.split('\n')[1].split('•')[1].replace("%","")))
beer_deets['rb_overall_score'] = beer_deets.return_string.apply(lambda x: x.split('\n')[2])
beer_deets['rb_style_score'] = beer_deets.return_string.apply(lambda x: x.split('\n')[3])
beer_deets['rb_user_rating'] = beer_deets.return_string.apply(lambda x: x.split('\n')[4])
beer_deets['rb_rating_number'] = beer_deets.return_string.apply(lambda x: x.split('\n')[5].replace("(","").replace(")",""))


In [76]:
# sorting the no alcohol measure
no_alcohol['name_found'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[0].strip())
no_alcohol['flag'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[1].split('•')[0].split(" ")[0])
no_alcohol['type'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[1].split('•')[0][2:])
no_alcohol['abv'] = no_alcohol.type.apply(lambda x: 0 if x[-1] == '0' else None)
no_alcohol['type'] = no_alcohol.type.apply(lambda x: x[:-1] if x[-1] == '0' else x)
no_alcohol['check'] = no_alcohol.return_string.apply(lambda x: len(x.split('\n')))
no_alcohol = no_alcohol[no_alcohol['check']>4].copy()
no_alcohol['rb_overall_score'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[2])
no_alcohol['rb_style_score'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[3])
no_alcohol['rb_user_rating'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[4])
no_alcohol['rb_rating_number'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[5].replace("(","").replace(")",""))
no_alcohol.drop('check',inplace=True, axis=1)

no_alcohol

,return_string,name_found,flag,type,abv,rb_overall_score,rb_style_score,rb_user_rating,rb_rating_number
53,MacKinnon Brothers Red Fox ale\n🇨🇦 Flavored - ...,MacKinnon Brothers Red Fox ale,🇨🇦,Flavored - Other,0.0,46,72,3.43,16
57,Granite Brewery Peculiar (Whiskey Barrel)\n🇨🇦 ...,Granite Brewery Peculiar (Whiskey Barrel),🇨🇦,Old Ale,0.0,57,43,3.39,15
98,Bellwoods Jelly King (Kiwi-Passionfruit)\n🇨🇦 S...,Bellwoods Jelly King (Kiwi-Passionfruit),🇨🇦,Sour / Wild Beer - Flavored,NaN,-,-,3.87,6
113,Peace Tree Cedar Ridge Coffee Stout\n🇺🇸 Stout ...,Peace Tree Cedar Ridge Coffee Stout,🇺🇸,Stout - Flavored,NaN,-,-,3.80,1
115,8e Péché Stout Incendiaire (Bourbon & Rye)\n🇨🇦...,8e Péché Stout Incendiaire (Bourbon & Rye),🇨🇦,Stout - Imperial Flavored,NaN,-,-,3.90,3
159,Granite - Toronto Le Cocoa Cherry\n🇨🇦 Stout - ...,Granite - Toronto Le Cocoa Cherry,🇨🇦,Stout - Flavored,NaN,-,-,3.20,1
161,Granite -Toronto Le Cocoa Chile\n🇨🇦 Stout - Fl...,Granite -Toronto Le Cocoa Chile,🇨🇦,Stout - Flavored,NaN,-,-,3.60,1
173,Counterpart Daybreak\n🇨🇦 Stout - Flavored\n-\n...,Counterpart Daybreak,🇨🇦,Stout - Flavored,NaN,-,-,3.40,1
178,Rock House When I Whip You Whip Pineapple Whip...,Rock House When I Whip You Whip Pineapple Whip,🇺🇸,Gose - Flavored,NaN,-,-,3.50,1
208,Harpoon Baltic Porter BA Whistlepig\n🇺🇸 Porter...,Harpoon Baltic Porter BA Whistlepig,🇺🇸,Porter - Flavored,NaN,-,-,3.80,1


In [77]:
beer_df = pd.concat([beer_df,beer_deets,no_alcohol])

In [78]:
beer_df

,return_string,flag,type,abv,rb_overall_score,rb_style_score,rb_user_rating,rb_rating_number,name_found
0,St. Peter's Ruby Red Ale\n🇬🇧 Bitter - Ordinary...,🇬🇧,Bitter - Ordinary / Best,4.3,49,95,3.25,415,St. Peter's Ruby Red Ale
1,Shepherd Neame Spitfire (Bottle)\n🇬🇧 Bitter - ...,🇬🇧,Bitter - Ordinary / Best,4.5,34,59,3.05,1083,Shepherd Neame Spitfire (Bottle)
2,Arbor Rocketman\n🇬🇧 IPA • 6.0%\n95\n97\n3.74\n...,🇬🇧,IPA,6.0,95,97,3.74,59,Arbor Rocketman
3,Shepherd Neame India Pale Ale (Bottle)\n🇬🇧 IPA...,🇬🇧,IPA - English,6.1,46,47,3.22,420,Shepherd Neame India Pale Ale (Bottle)
4,Shepherd Neame Christmas Ale\n🇬🇧 Strong Ale - ...,🇬🇧,Strong Ale - English,7.0,45,46,3.18,409,Shepherd Neame Christmas Ale
...,...,...,...,...,...,...,...,...,...
210,MERIT Expresso Porter\n🇨🇦 Porter - Flavored\n-...,🇨🇦,Porter - Flavored,NaN,-,-,3.40,1,MERIT Expresso Porter
216,Niagara College Teaching White Stout\n🇨🇦 Stout...,🇨🇦,Stout - Milk / Sweet,NaN,-,-,3.00,1,Niagara College Teaching White Stout
221,5 Paddles Mike's Special Danger\n🇨🇦 Stout - Fl...,🇨🇦,Stout - Flavored,NaN,-,-,3.50,1,5 Paddles Mike's Special Danger
227,Ceria Indiewave (no THC)\n🇺🇸 Low / No Alcohol ...,🇺🇸,Low / No Alcohol Beer - Pale,NaN,-,-,3.33,3,Ceria Indiewave (no THC)


In [79]:
# compare to the unique list to create a new to-be-scraped list
not_found_beer = df.beer_name[~df.beer_name.isin(beer_df.name_found)].unique()
print("There are", len(not_found_beer)-len(not_found_df), "new beers that were not found or returned errors. They will be added to the no search list.")
not_found_df = pd.concat([not_found_df,pd.DataFrame(not_found_beer,columns = ["not_found"])])
not_found_df.drop_duplicates(inplace=True)

There are 8 new beers that were not found or returned errors. They will be added to the no search list.


In [80]:
beer_df.to_csv('beer_deets.csv',index=False)
not_found_df.to_csv("not_found.csv", index=False)